# 04. Clustering and Visualization (E. coli Analysis)
### PCA and Differential Expression Overview

---

## 1. Overview
In this notebook, we analyze the normalized *E. coli* data to understand the biological differences between **WT**, **Mutant**, and **Biofilm** conditions.
Key steps:
1.  **Feature Selection:** Identify highly variable genes.
2.  **Scaling:** Prepare data for dimensionality reduction.
3.  **PCA:** Visualize sample grouping.
4.  **Heatmap:** View top variable genes.

In [ ]:
# 1. Load Libraries
library(Seurat)
library(dplyr)
library(tximport)
library(ggplot2)

print("✅ Libraries Loaded.")

# 2. Re-Load the E. coli Data
# (Since we are in a new notebook, we need to load the data again)

valid_samples <- c("SRR25445875", "SRR25445876", "SRR25445877", "SRR25445878", "SRR25445879", "SRR25445880")
files <- file.path("../processed_data/salmon_quant", valid_samples, "quant.sf")
names(files) <- valid_samples

# Import Data
txi <- tximport(files, type = "salmon", txOut = TRUE)
counts <- txi$counts
rownames(counts) <- paste0("MG1655_", rownames(counts))

# Define Metadata again
metadata <- data.frame(
  sample_id = valid_samples,
  species = rep("E. coli", 6),
  condition = c("WT", "WT", "Mutant", "Mutant", "Mutant_Biofilm", "Mutant_Biofilm"),
  replicate = rep(c("Rep1", "Rep2"), 3)
)
rownames(metadata) <- valid_samples

# Create Seurat Object & Normalize
bact_seurat <- CreateSeuratObject(counts = counts, meta.data = metadata, project = "E_coli_Analysis")
bact_seurat <- NormalizeData(bact_seurat, normalization.method = "LogNormalize", scale.factor = 10000)

print("✅ Data Reloaded & Ready for Analysis in Notebook 04!")

In [ ]:
# 2. Identification of Variable Features
# We select the top 2000 genes that show high variability between samples
bact_seurat <- FindVariableFeatures(bact_seurat, selection.method = "vst", nfeatures = 2000)

# 3. Scaling the Data
# This centers the data (Mean = 0, Variance = 1) so high-expression genes don't dominate
all.genes <- rownames(bact_seurat)
bact_seurat <- ScaleData(bact_seurat, features = all.genes)

print("✅ Data Scaled and Variable Features Identified.")

In [ ]:
# 3. Run PCA 
# We set npcs = 5 because we only have 6 samples
print("🧮 Running PCA...")
bact_seurat <- RunPCA(bact_seurat, 
                      features = VariableFeatures(object = bact_seurat), 
                      npcs = 5) # <--- الحل هنا

# 4. Visualize PCA
# We visualize the first two components (PC1 and PC2)
print("🎨 Plotting PCA...")
p1 <- DimPlot(bact_seurat, reduction = "pca", group.by = "condition", pt.size = 5) + 
  ggtitle("PCA Analysis: E. coli Conditions") +
  theme(plot.title = element_text(hjust = 0.5))

print(p1)

In [ ]:
# 5. Differential Expression (The "ROC" Method for Small Samples)

# 1. Set Identity
Idents(bact_seurat) <- "condition"

print("🕵️‍♂️ Finding markers using ROC test (No p-values needed)...")

# 2. Find Markers using ROC
# This calculates "Classification Power" instead of P-values.
# return.thresh = 0.5 means "Any gene better than random guessing"
bact.markers <- FindAllMarkers(bact_seurat, 
                               test.use = "roc",       # <--- The Fix
                               only.pos = TRUE, 
                               min.pct = 0.1, 
                               logfc.threshold = 0.1,
                               return.thresh = 0.5,    # Low threshold to ensure results
                               verbose = FALSE)

# 3. Check results
if(nrow(bact.markers) == 0) {
    # PLAN B: If even ROC fails, we just plot the top Variable Genes (Guaranteed to exist)
    print("⚠️ ROC found nothing (very rare). Switching to Top Variable Features...")
    top_genes <- head(VariableFeatures(bact_seurat), 20)
    
    DoHeatmap(bact_seurat, features = top_genes) + 
      ggtitle("Top Variable Genes (No Stats)") +
      theme(text = element_text(size = 12))
      
} else {
    print(paste("✅ Found", nrow(bact.markers), "markers using ROC!"))
    
    # 4. Get Top 10 genes by "myAUC" (Area Under Curve - classification power)
    top10 <- bact.markers %>% 
                group_by(cluster) %>% 
                top_n(n = 10, wt = myAUC)

    # 5. Generate Heatmap
    print("🔥 Generating Heatmap...")
    p <- DoHeatmap(bact_seurat, features = top10$gene) + 
      ggtitle("Top Gene Expression (ROC Analysis)") +
      theme(text = element_text(size = 12))
      
    print(p)
}